#### HW 15.2

In [82]:
# Loading the library
from pulp import *
import pandas as pd

In [83]:
# Loading the data
df = pd.read_excel("diet.xls")

### Question 1

**1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using `PuLP`.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)**

#### Step 1: Loading the data 

First, we need to load the data. Here we will use only until the 63th row, as these rows are containing the variables and the values.

In [84]:
## Change each row into list 
diet_df = df[0:64]
diet_df = diet_df.values.tolist()

In [85]:
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


#### Step 2: Modifying the format of the data for variables 

In this part, we transform the information from the dataframe into dictionary for the necessary variables.
We need three variables: 1) the amount of food, 2) the amount of nutrients per unit of food, and 3) the per-unit cost of food. 
Here, I saved the variables as a dictionary format. For easy calculation, I combined the dictionaries of the nutrients into a list, so that we can access the values like a matrix.

In [86]:
foods = [x[0] for x in diet_df]
calories = dict([(x[0], float(x[3])) for x in diet_df])
cholesterol = dict([(x[0], float(x[4])) for x in diet_df])
total_fat = dict([(x[0], float(x[5])) for x in diet_df])
sodium = dict([(x[0], float(x[6])) for x in diet_df])
carbohydrates = dict([(x[0], float(x[7])) for x in diet_df])
dietary_fiber = dict([(x[0], float(x[8])) for x in diet_df])
protein = dict([(x[0], float(x[9])) for x in diet_df])
vitamin_a = dict([(x[0], float(x[10])) for x in diet_df])
vitamin_c = dict([(x[0], float(x[11])) for x in diet_df])
calcium = dict([(x[0], float(x[12])) for x in diet_df])
iron = dict([(x[0], float(x[13])) for x in diet_df])

## Append the dictionary from calories to iron
B = []
for i in range(0,11):
    B.append(dict([(x[0], float(x[i+3])) for x in diet_df]))
    
## Cost data
cost = dict([(x[0], float(x[1])) for x in diet_df])

The minimum and maximum intake values were saved in a list to be used in the constraints. 

In [87]:
# Create limits: from calories to iron 
min_take = [1500, 30, 20, 800, 130, 125, 60, 1000, 400, 700, 10]
max_take = [2500, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]

#### Step 3: Initializing the problem class

In [88]:
## Initializing the problem class - minimizing 
model1 = LpProblem("Optimized_diet_problem", LpMinimize)

#### Step 4: Setting the variables

In [89]:
## Foods variables as continuous 
foods_var = LpVariable.dicts("foods",foods, 0) # .dicts(name, indexs, lowBound=None, upBound=None, cat=0, indexStart=[])

In [90]:
## Chosen variable for each food
choice_var = LpVariable.dicts("chosen", foods, 0, 1, "Binary")

#### Step 5: Define objective function 

As our objective is to minimize the expense for the diet that satisfies the daily nutrients intake, we set the objective function as the sum of the cost for the foods. 

In [92]:
## Define the objective functions 
model1 += lpSum(cost[j]*foods_var[j] for j in foods)

#### Step 6: Add contraints

In [93]:
## Add constraints for minimum and maximum intake of nutrients

for i in range(0, 11): 
    B_x = pulp.lpSum([B[i][j] * foods_var[j] for j in foods]) # the total amount of nutrients intake 
    condition1 = min_take[i] <= + B_x
    model1 += condition1
    
for i in range(0, 11):
    B_x = pulp.lpSum([B[i][j] * foods_var[j] for j in foods]) # the total amount of nutrients intake 
    condition2 = max_take[i] >= + B_x
    model1 += condition2

#### Step 7: Solving the optimization problem 

In [94]:
## Before solving the optimization, review the model built so far
print(model1)

Optimized_diet_problem:
MINIMIZE
0.23*foods_2%_Lowfat_Milk + 0.16*foods_3.3%_Fat,Whole_Milk + 0.24*foods_Apple,Raw,W_Skin + 0.16*foods_Apple_Pie + 0.16*foods_Bagels + 0.15*foods_Banana + 0.67*foods_Beanbacn_Soup,W_Watr + 0.15*foods_Bologna,Turkey + 0.05*foods_Butter,Regular + 0.31*foods_Cap'N_Crunch + 0.07*foods_Carrots,Raw + 0.04*foods_Celery,_Raw + 0.25*foods_Cheddar_Cheese + 0.28*foods_Cheerios + 0.39*foods_Chicknoodl_Soup + 0.03*foods_Chocolate_Chip_Cookies + 0.28*foods_Corn_Flks,_Kellogg'S + 0.39*foods_Couscous + 0.65*foods_Crm_Mshrm_Soup,W_Mlk + 0.27*foods_Frankfurter,_Beef + 0.16*foods_Frozen_Broccoli + 0.18*foods_Frozen_Corn + 0.32*foods_Grapes + 0.33*foods_Ham,Sliced,Extralean + 0.83*foods_Hamburger_W_Toppings + 0.31*foods_Hotdog,_Plain + 0.15*foods_Kielbasa,Prk + 0.49*foods_Kiwifruit,Raw,Fresh + 0.02*foods_Lettuce,Iceberg,Raw + 0.17*foods_Macaroni,Ckd + 0.52*foods_Malt_O_Meal,Choc + 0.99*foods_New_E_Clamchwd,W_Mlk + 0.75*foods_Neweng_Clamchwd + 0.82*foods_Oatmeal + 0.09*foods

In [95]:
# The status 1 meaning it has the optimal solution 
model1.solve()

1

In [96]:
print('Optimization Solution:')
for v in model1.variables():
    if v.varValue > 0:
        if str(v).find('Chosen'):
            print(str(v.varValue) + " units of " + str(v))

Optimization Solution:
52.64371 units of foods_Celery,_Raw
0.25960653 units of foods_Frozen_Broccoli
63.988506 units of foods_Lettuce,Iceberg,Raw
2.2929389 units of foods_Oranges
0.14184397 units of foods_Poached_Eggs
13.869322 units of foods_Popcorn,Air_Popped


In [97]:
print("Total cost of food = $%.2f" % value(model1.objective))

Total cost of food = $4.34


### Question 2

**2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:**<br><br>
a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)<br>
b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.<br>
c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]


#### Step 1: Initializing and defining the second optimization model 

In [52]:
## Initializing the problem class - minimizing 
model2 = LpProblem("Optimized_diet_problem2", LpMinimize)

In [53]:
## Defining the objective function 
model2 += lpSum(cost[j]*foods_var[j] for j in foods)

#### Step 2: Adding new constraints to the model 

In this part, I added the new constraints: 1) the amount of food greater than 0.1 unit, 2) celery and frozen broccoli at most one not both, and 3) at least three kinds of meat/poultry/fish/eggs. 

In [57]:
## Add constraint for the amount of food: should be greater than 0.1, less than 1000 (an arbitrary large number)
for j in foods: 
    model2 += foods_var[j] <= 1000 * choice_var[j]
    model2 += foods_var[j] >= 0.1 * choice_var[j]

In [59]:
## Add constraint for the amount of nutrients 

for i in range(0, 11): 
    B_x = pulp.lpSum([B[i][j] * foods_var[j] for j in foods]) # the total amount of nutrients intake 
    condition1 = min_take[i] <= + B_x
    model2 += condition1
    
for i in range(0, 11):
    B_x = pulp.lpSum([B[i][j] * foods_var[j] for j in foods]) # the total amount of nutrients intake 
    condition2 = max_take[i] >= + B_x
    model2 += condition2

In [61]:
## Add constraint for celery and frozen broccoli at most one, but not both

model2 += choice_var['Frozen Broccoli'] + choice_var['Celery, Raw'] <= 1

In [70]:
### Add constraint for at least 3 kinds of meat/poultry/fish/eggs

model2 += choice_var['Roasted Chicken'] + choice_var['Poached Eggs'] + choice_var['Scrambled Eggs'] + choice_var['Bologna,Turkey']+ choice_var['Frankfurter, Beef']+ choice_var['Ham,Sliced,Extralean']+ choice_var['Kielbasa,Prk']+ choice_var['Hamburger W/Toppings']+ choice_var['Hotdog, Plain']+ choice_var['Pork']+ choice_var['Sardines in Oil']+ choice_var['White Tuna in Water'] >= 3

#### Step 3: Solving the optimization problem 

In [71]:
### review model 2
model2

Optimized_diet_problem2:
MINIMIZE
0.23*foods_2%_Lowfat_Milk + 0.16*foods_3.3%_Fat,Whole_Milk + 0.24*foods_Apple,Raw,W_Skin + 0.16*foods_Apple_Pie + 0.16*foods_Bagels + 0.15*foods_Banana + 0.67*foods_Beanbacn_Soup,W_Watr + 0.15*foods_Bologna,Turkey + 0.05*foods_Butter,Regular + 0.31*foods_Cap'N_Crunch + 0.07*foods_Carrots,Raw + 0.04*foods_Celery,_Raw + 0.25*foods_Cheddar_Cheese + 0.28*foods_Cheerios + 0.39*foods_Chicknoodl_Soup + 0.03*foods_Chocolate_Chip_Cookies + 0.28*foods_Corn_Flks,_Kellogg'S + 0.39*foods_Couscous + 0.65*foods_Crm_Mshrm_Soup,W_Mlk + 0.27*foods_Frankfurter,_Beef + 0.16*foods_Frozen_Broccoli + 0.18*foods_Frozen_Corn + 0.32*foods_Grapes + 0.33*foods_Ham,Sliced,Extralean + 0.83*foods_Hamburger_W_Toppings + 0.31*foods_Hotdog,_Plain + 0.15*foods_Kielbasa,Prk + 0.49*foods_Kiwifruit,Raw,Fresh + 0.02*foods_Lettuce,Iceberg,Raw + 0.17*foods_Macaroni,Ckd + 0.52*foods_Malt_O_Meal,Choc + 0.99*foods_New_E_Clamchwd,W_Mlk + 0.75*foods_Neweng_Clamchwd + 0.82*foods_Oatmeal + 0.09*food

In [72]:
model2.solve()

1

In [79]:
print('Optimization Solution:')

for v in model2.variables():
    if v.varValue > 0:
        if str(v).find('Chosen'):
            print(str(v.varValue) + " units of " + str(v))

Optimization Solution:
1.0 units of chosen_Celery,_Raw
1.0 units of chosen_Kielbasa,Prk
1.0 units of chosen_Lettuce,Iceberg,Raw
1.0 units of chosen_Oranges
1.0 units of chosen_Peanut_Butter
1.0 units of chosen_Poached_Eggs
1.0 units of chosen_Popcorn,Air_Popped
1.0 units of chosen_Scrambled_Eggs
42.399358 units of foods_Celery,_Raw
0.1 units of foods_Kielbasa,Prk
82.802586 units of foods_Lettuce,Iceberg,Raw
3.0771841 units of foods_Oranges
1.9429716 units of foods_Peanut_Butter
0.1 units of foods_Poached_Eggs
13.223294 units of foods_Popcorn,Air_Popped
0.1 units of foods_Scrambled_Eggs


In [80]:
print("Total cost of food = $%.2f" % value(model2.objective))

Total cost of food = $4.51
